In [1]:
import numpy as np
from qiskit import execute, QuantumCircuit
from qiskit.test.mock import FakeParis
from qiskit.providers.aer import QasmSimulator, noise

from qiskit.ignis.verification.tomography import marginal_counts, expectation_counts
from qiskit.ignis.mitigation.measurement import (
    MeasMitigatorGenerator,
    CompleteMeasMitigator,
    TensoredMeasMitigator,
    CTMPMeasMitigator,
    fit_complete_meas_mitigator,
    fit_tensored_meas_mitigator,
    fit_ctmp_meas_mitigator,
    counts_expectation_value)

## Sim noise model

In [2]:
sim = QasmSimulator()

# Example max qubit number
num_qubits = 4

# Create readout errors
readout_errors = []
for i in range(num_qubits):
    p_error1 = (i + 1) * 0.02
    p_error0 = 2 * p_error1
    ro_error = noise.ReadoutError([[1 - p_error0, p_error0], [p_error1, 1 - p_error1]])
    readout_errors.append(ro_error)

# Readout Error only
nm = noise.NoiseModel()
for i in range(num_qubits):
    nm.add_readout_error(readout_errors[i], [i])
seed_simulator = 100

### Cal (Full)

Ideal cal with only readout error

In [3]:
circuits1, metadata1, _ = MeasMitigatorGenerator(num_qubits, method='complete').run()

job_cal1 = execute(circuits1, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal1 = job_cal1.result()

In [4]:
complete_mitigator = fit_complete_meas_mitigator(result_cal1, metadata1)
print('Mitigation Overhead:', complete_mitigator.mitigation_overhead())
print('Required Shots (delta = 0.05):', complete_mitigator.required_shots(0.05))

Mitigation Overhead: 6
Required Shots (delta = 0.05): 8911


### Cal (Tensor)

In [5]:
circuits2, metadata2, _ = MeasMitigatorGenerator(num_qubits, method='tensored').run()

job_cal2 = execute(circuits2, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal2 = job_cal2.result()

In [6]:
tensored_mitigator = fit_tensored_meas_mitigator(result_cal2, metadata2)
print('Mitigation Overhead:', tensored_mitigator.mitigation_overhead())
print('Required Shots (delta = 0.05):', tensored_mitigator.required_shots(0.05))

Mitigation Overhead: 6
Required Shots (delta = 0.05): 8962


### Cal (CTMP)

In [7]:
circuits3, metadata3, _ = MeasMitigatorGenerator(num_qubits, method='CTMP').run()

job_cal3 = execute(circuits3, sim,
                   seed_simulator=seed_simulator,
                   shots=10000,
                   noise_model=nm,
                   backend_options={'method': 'density_matrix'})
result_cal3 = job_cal3.result()

In [8]:
ctmp_mitigator = fit_ctmp_meas_mitigator(result_cal3, metadata3)
print('Mitigation Overhead:', ctmp_mitigator.mitigation_overhead())
print('Required Shots (delta = 0.05):', ctmp_mitigator.required_shots(0.05))

Mitigation Overhead: 7
Required Shots (delta = 0.05): 9766


### Expval (all-qubits)

In [9]:
# Test Circuit Measure <XXXX> stabilizer
qc = QuantumCircuit(4, 4)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)
qc.h(range(4))
qc.measure(range(4), range(4))

# Simulation
shots = 10000
result_target = execute(qc, sim, shots=shots).result()
result_noise = execute(qc, sim, shots=shots, noise_model=nm).result()
counts_target = result_target.get_counts(0)
counts_noise = result_noise.get_counts(0)

# Expvals
expvals = [
    ('Target', counts_expectation_value(counts_target)),
    ('Unmitigated', counts_expectation_value(counts_noise)),
    ('Mitigated (Complete)', complete_mitigator.expectation_value(counts_noise)),
    ('Mitigated (Tensored)', tensored_mitigator.expectation_value(counts_noise)),
    ('Mitigated (CTMP)', ctmp_mitigator.expectation_value(counts_noise)),
]
print('\nExpectation Values')
for label, expval in expvals:
    print('- {}: {:f}'.format(label, expval))


Expectation Values
- Target: 1.000000
- Unmitigated: 0.508000
- Mitigated (Complete): 0.983112
- Mitigated (Tensored): 0.988800
- Mitigated (CTMP): 1.011281


### Expval (all-qubits, permuted)

NOTE: This is not yet implemented for CTMP method

In [10]:
for meas_qubits in [[0, 1, 2, 3], [0, 1, 3, 2], [1, 3, 0, 2], [3, 0, 1, 2], [2, 0, 3, 1], [3, 2, 1, 0]]:

    # Test Circuit Measure <XXXX> stabilizer
    qc = QuantumCircuit(4, 4)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    qc.h(range(4))
    qc.measure(meas_qubits, range(4))

    # Simulation
    shots = 10000

    result_target = execute(qc, sim, shots=shots, seed_simulator=seed_simulator).result()
    result_noise = execute(qc, sim, shots=shots,seed_simulator=seed_simulator,
                           noise_model=nm).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expvals = [
        ('Target', counts_expectation_value(counts_target)),
        ('Unmitigated', counts_expectation_value(counts_noise)),
        ('Mitigated (Complete)', complete_mitigator.expectation_value(counts_noise, qubits=meas_qubits)),
        ('Mitigated (Tensored)', tensored_mitigator.expectation_value(counts_noise, qubits=meas_qubits)),
        ('Mitigated (CTMP)', ctmp_mitigator.expectation_value(counts_noise)),
    ]
    print('\nExpectation Values (clbits: {})'.format(meas_qubits))
    for label, expval in expvals:
        print('- {}: {:f}'.format(label, expval))


Expectation Values (clbits: [0, 1, 2, 3])
- Target: 1.000000
- Unmitigated: 0.528400
- Mitigated (Complete): 1.023113
- Mitigated (Tensored): 1.029068
- Mitigated (CTMP): 1.033080

Expectation Values (clbits: [0, 1, 3, 2])
- Target: 1.000000
- Unmitigated: 0.519600
- Mitigated (Complete): 1.005603
- Mitigated (Tensored): 1.011569
- Mitigated (CTMP): 1.032680

Expectation Values (clbits: [1, 3, 0, 2])
- Target: 1.000000
- Unmitigated: 0.524600
- Mitigated (Complete): 1.016037
- Mitigated (Tensored): 1.022079
- Mitigated (CTMP): 1.047280

Expectation Values (clbits: [3, 0, 1, 2])
- Target: 1.000000
- Unmitigated: 0.522000
- Mitigated (Complete): 1.012298
- Mitigated (Tensored): 1.018193
- Mitigated (CTMP): 1.015880

Expectation Values (clbits: [2, 0, 3, 1])
- Target: 1.000000
- Unmitigated: 0.516000
- Mitigated (Complete): 1.001812
- Mitigated (Tensored): 1.007599
- Mitigated (CTMP): 1.031280

Expectation Values (clbits: [3, 2, 1, 0])
- Target: 1.000000
- Unmitigated: 0.518400
- Mitigat

### Expval Partial Qubits

NOTE: This is not yet implemented for CTMP method

In [11]:
for qubits in [[0, 1], [0, 2], [0, 3], [3, 1], [2, 1], [3, 2], [2, 3]]:
    
    # Test Circuit
    qc = QuantumCircuit(4, 2)
    qc.h(qubits[0])
    qc.cx(qubits[0], qubits[1])
    qc.measure(qubits, [0, 1])

    # Simulation
    shots = 8000
    seed = 1101
    result_target = execute(qc, sim, shots=shots, seed_simulator=seed_simulator).result()
    result_noise = execute(qc, sim, shots=shots,seed_simulator=seed_simulator,
                           noise_model=nm).result()
    counts_target = result_target.get_counts(0)
    counts_noise = result_noise.get_counts(0)

    # Expvals
    expvals = [
        ('Target', counts_expectation_value(counts_target)),
        ('Unmitigated', counts_expectation_value(counts_noise)),
        ('Mitigated (Complete)', complete_mitigator.expectation_value(counts_noise, qubits=qubits)),
        ('Mitigated (Tensored)', tensored_mitigator.expectation_value(counts_noise, qubits=qubits)),
        ('Mitigated (CTMP)', ctmp_mitigator.expectation_value(counts_noise)),
    ]
    print('\nExpectation Values (qubits: {})'.format(qubits))
    for label, expval in expvals:
        print('- {}: {:f}'.format(label, expval))


Expectation Values (qubits: [0, 1])
- Target: 1.000000
- Unmitigated: 0.827000
- Mitigated (Complete): 0.995536
- Mitigated (Tensored): 0.995288
- Mitigated (CTMP): 1.415723

Expectation Values (qubits: [0, 2])
- Target: 1.000000
- Unmitigated: 0.765250
- Mitigated (Complete): 0.992570
- Mitigated (Tensored): 0.998213
- Mitigated (CTMP): 1.281975

Expectation Values (qubits: [0, 3])
- Target: 1.000000
- Unmitigated: 0.712000
- Mitigated (Complete): 0.996379
- Mitigated (Tensored): 0.999863
- Mitigated (CTMP): 1.206727

Expectation Values (qubits: [3, 1])
- Target: 1.000000
- Unmitigated: 0.668250
- Mitigated (Complete): 0.993435
- Mitigated (Tensored): 0.993098
- Mitigated (CTMP): 1.133728

Expectation Values (qubits: [2, 1])
- Target: 1.000000
- Unmitigated: 0.723000
- Mitigated (Complete): 1.000481
- Mitigated (Tensored): 0.998830
- Mitigated (CTMP): 1.219227

Expectation Values (qubits: [3, 2])
- Target: 1.000000
- Unmitigated: 0.617500
- Mitigated (Complete): 0.991132
- Mitigated 